In [3]:
!pip install transformers
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from transformers import pipeline
file_path = '/content/drive/MyDrive/Colab Notebooks/book_text.txt'
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english")

In [ ]:
# Read the book text
with open(file_path, 'r') as f:
    data = f.read()
chapters = data.split('CHAPTER')[1:]

chapter_entity_counts = {}

for chapter_num, chapter in enumerate(chapters, start=1):
    person_place = {}
    for paragraph in chapter.split('\n'):
        ner_results = ner(paragraph)
        
        i = 0
        while i < len(ner_results):
            entity = ner_results[i]['entity']
            if entity in ['I-PER', 'I-LOC']:
                word = ner_results[i]['word']
                
                # If the next word is of the same entity type, consider it as a part of the current entity
                while i+1 < len(ner_results) and ner_results[i+1]['entity'] == entity:
                    i += 1
                    word += " " + ner_results[i]['word']
                
                if word not in person_place:
                    person_place[word] = 0

                person_place[word] += 1
                
            i += 1

    chapter_entity_counts[chapter_num] = person_place

# Print the dictionary of chapters, person, place and counts
for chapter, entities in chapter_entity_counts.items():
    print(f"Chapter {chapter}:")
    for entity, count in entities.items():
        print(f"    {entity}: mentioned {count} times")

# Open a file in write mode
with open('entity_counts.txt', 'w') as f:
    for chapter, entities in chapter_entity_counts.items():
        f.write(f"Chapter {chapter}:\n")
        for entity, count in entities.items():
            f.write(f"    {entity}: mentioned {count} times\n")


Chapter 1:
    Europe America: mentioned 1 times
    Governor Higgins: mentioned 1 times
    Australia: mentioned 1 times
    Baker: mentioned 1 times
    Higgins ##on: mentioned 2 times
    Pacific Ocean: mentioned 1 times
    Columbus: mentioned 2 times
    West India Pacific: mentioned 1 times
    Atlantic: mentioned 1 times
    United States Europe: mentioned 1 times
    Al ##eu ##tian Ku ##lam ##ma ##k Um ##gu ##lli ##ch: mentioned 1 times
    Norman: mentioned 1 times
    Fi ##tz - James: mentioned 1 times
    England America Germany: mentioned 1 times
    Aristotle P ##lin ##y: mentioned 1 times
    Pont ##op ##pid ##an: mentioned 1 times
    Paul He ##gged ##e: mentioned 1 times
    Harrington: mentioned 1 times
    Archipelago: mentioned 1 times
    A Mo ##ign ##o: mentioned 1 times
    Peter ##mann: mentioned 1 times
    France: mentioned 1 times
    Lin ##n ##æ ##us: mentioned 1 times
    Hip ##pol ##yt ##us: mentioned 1 times
    Montreal Ocean: mentioned 1 times
    Canada

In [ ]:
from collections import defaultdict
# Read the book text
with open(file_path, 'r') as f:
    data = f.read()
chapters = data.split('CHAPTER')[1:]

chapter_entity_counts = {}

for chapter_num, chapter in enumerate(chapters, start=1):
    words = chapter.split()
    
    # Breaking chapter into 200 word segments
    for i in range(0, len(words), 200):
        segment = " ".join(words[i:i+200])
        ner_results = ner(segment)
        
        person_place = {'I-PER': defaultdict(int), 'I-LOC': defaultdict(int)}
        for result in ner_results:
            entity = result['entity']
            word = result['word']
            if entity in ['I-PER', 'I-LOC']:
                person_place[entity][word] += 1

        # Check if both person and place are present in this segment
        if person_place['I-PER'] and person_place['I-LOC']:
            if chapter_num not in chapter_entity_counts:
                chapter_entity_counts[chapter_num] = []
            chapter_entity_counts[chapter_num].append(person_place)

# Print the dictionary of chapters, person, place and counts
for chapter, segments in chapter_entity_counts.items():
    print(f"Chapter {chapter}:")
    for seg_num, entities in enumerate(segments, start=1):
        print(f"    Segment {seg_num}:")
        for entity_type, counts in entities.items():
            entity_label = "Person" if entity_type == 'I-PER' else "Place"
            for entity, count in counts.items():
                print(f"        {entity_label} '{entity}': mentioned {count} times")

# Open a file in write mode
with open('entity_counts_200_words.txt', 'w') as f:
    for chapter, segments in chapter_entity_counts.items():
        f.write(f"Chapter {chapter}:\n")
        for seg_num, entities in enumerate(segments, start=1):
            f.write(f"    Segment {seg_num}:\n")
            for entity_type, counts in entities.items():
                entity_label = "Person" if entity_type == 'I-PER' else "Place"
                for entity, count in counts.items():
                    f.write(f"        {entity_label} '{entity}': mentioned {count} times\n")


Chapter 1:
    Segment 1:
        Person 'Governor': mentioned 2 times
        Person 'Higgins': mentioned 1 times
        Person 'Baker': mentioned 1 times
        Place 'Australia': mentioned 1 times
    Segment 2:
        Person 'Governor': mentioned 1 times
        Person 'Higgins': mentioned 1 times
        Person '##on': mentioned 1 times
        Place 'Pacific': mentioned 1 times
        Place 'Ocean': mentioned 1 times
        Place 'Atlantic': mentioned 1 times
        Place 'United': mentioned 1 times
        Place 'States': mentioned 1 times
        Place 'Europe': mentioned 1 times
    Segment 3:
        Person 'Fi': mentioned 1 times
        Person '##tz': mentioned 1 times
        Person '-': mentioned 1 times
        Person 'James': mentioned 1 times
        Place 'Al': mentioned 1 times
        Place '##eu': mentioned 1 times
        Place '##tian': mentioned 1 times
        Place 'Ku': mentioned 1 times
        Place '##lam': mentioned 1 times
        Place '##ma': men

In [ ]:
# Read the book text
with open(file_path, 'r') as f:
    data = f.read()
chapters = data.split('CHAPTER')[1:]

chapter_entity_counts = {}

for chapter_num, chapter in enumerate(chapters, start=1):
    person_dict = {}
    place_dict = {}

    for paragraph in chapter.split('\n'):
        ner_results = ner(paragraph)
        
        i = 0
        while i < len(ner_results):
            entity = ner_results[i]['entity']
            word = ner_results[i]['word']
            
            # If the next word is of the same entity type, consider it as a part of the current entity
            while i+1 < len(ner_results) and ner_results[i+1]['entity'] == entity:
                i += 1
                word += " " + ner_results[i]['word']
            
            # Check if it's a person or a location
            if entity in ['B-PER', 'I-PER']:
                if word not in person_dict:
                    person_dict[word] = 0
                person_dict[word] += 1
            elif entity in ['B-LOC', 'I-LOC']:
                if word not in place_dict:
                    place_dict[word] = 0
                place_dict[word] += 1

            i += 1

    chapter_entity_counts[chapter_num] = {'Person': person_dict, 'Place': place_dict}

# Print the dictionary of chapters, person, place and counts
for chapter, entities in chapter_entity_counts.items():
    print(f"Chapter {chapter}:")
    for entity_type, entity_dict in entities.items():
        print(f"  {entity_type}s:")
        for entity, count in entity_dict.items():
            print(f"    {entity}: mentioned {count} times")

# Open a file in write mode
with open('entity_counts_person_place.txt', 'w') as f:
    for chapter, entities in chapter_entity_counts.items():
        f.write(f"Chapter {chapter}:\n")
        for entity_type, entity_dict in entities.items():
            f.write(f"  {entity_type}s:\n")
            for entity, count in entity_dict.items():
                f.write(f"    {entity}: mentioned {count} times\n")


Chapter 1:
  Persons:
    Governor Higgins: mentioned 1 times
    Baker: mentioned 1 times
    Higgins ##on: mentioned 2 times
    Columbus: mentioned 2 times
    Fi ##tz - James: mentioned 1 times
    Aristotle P ##lin ##y: mentioned 1 times
    Pont ##op ##pid ##an: mentioned 1 times
    Paul He ##gged ##e: mentioned 1 times
    Harrington: mentioned 1 times
    A Mo ##ign ##o: mentioned 1 times
    Peter ##mann: mentioned 1 times
    Lin ##n ##æ ##us: mentioned 1 times
    Hip ##pol ##yt ##us: mentioned 1 times
    Anderson: mentioned 3 times
  Places:
    Europe America: mentioned 1 times
    Australia: mentioned 1 times
    Pacific Ocean: mentioned 1 times
    West India Pacific: mentioned 1 times
    Atlantic: mentioned 1 times
    United States Europe: mentioned 1 times
    Al ##eu ##tian Ku ##lam ##ma ##k Um ##gu ##lli ##ch: mentioned 1 times
    Norman: mentioned 1 times
    England America Germany: mentioned 1 times
    Archipelago: mentioned 1 times
    France: mentioned 1 t

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

chap_3_ground_truth_people = {'J. B. Hobson': 1, 'Secretary of Marine': 1, 'Conseil': 10,
                       'Commander Farragut': 4, 'Aronnax': 1, 
                       'Engineer': 1}
chap_3_ground_truth_place = {'North Sea': 1, 'Plantes': 1, 'Europe': 2, 'France': 4,
                             'North Pacific Ocean': 1, 'China': 1, 'Congo': 1, 
                             'Paris': 1, 'Abramham Lincoln (Ship)': 6, 'Brooklyn': 2,
                             'New York': 3, 'East River': 1, 'New Jersey': 1,
                             'Hudson': 1, 'Sandy Hook Point': 1, 'Long Island': 1,
                             'Fire Island': 1, 'Atlantic': 1}
chap_3_ground_truth = {
**chap_3_ground_truth_people, 
**chap_3_ground_truth_place}
chap_4_ground_truth_people = {
    'Captain Farragut': 6,
    'Knight of Rhodes': 1,
    'Dieudonné de Gozon': 1,
    'Ned Land': 15,
    'Aronnax': 2,
    'Rabelais': 1,
    'Homer': 1,
    'Canadian Homer': 1
}

chap_4_ground_truth_place = {
    'Rhodes': 1,
    'Pacific Ocean': 1,
    'Cape Blanc': 1,
    'Patagonia': 1,
    'Tropic of Capricorn': 1,
    'Straits of Magellan': 1,
    'Quebec': 1,
    'France': 1,
    'The North': 1,
    'Scotia': 1
}
chap_4_ground_truth = {
**chap_4_ground_truth_people, 
**chap_4_ground_truth_place}
chapter3 = {
    "J.B. Hobson": 1,
    "North Sea": 1,
    "Plantes": 1,
    "Europe": 2,
    "France": 3,
    "Museum": 1,
    "North Pacific Ocean": 1,
    "Conseil": 5,
    "China Congo": 1,
    "Babiroussa": 2,
    "Paris": 1,
    "Farragut": 6,
    "Aronnax": 1,
    "Abraham Lincoln (Ship)": 4,
    "Con": 1,
    "Brooklyn": 1,
    "Brooklyn New York": 1,
    "East River": 1,
    "Hudson": 1,
    "New York": 1,
    "New Jersey": 1,
    "Sandy Hook Point": 1,
    "New York Channel": 1,
    "Long Island": 1,
    "Fire Island": 1,
    "Atlantic": 1
}

chapter4 = {
    "Farragut": 3,
    "Knight": 1,
    "Rhodes": 1,
    "Dieudonné de Gozon": 1,
    "Captain Farragut": 1,
    "Abraham Lincoln": 2,
    "Pacific": 2,
    "Ned Land": 9,
    "Quebec": 1,
    "France": 1,
    "Homer": 1,
    "North": 1,
    "Ned": 13,
    "Cape Blanc": 1,
    "Patagonia": 1,
    "Tropic of Capricorn Straits of Magellan": 1,
    "Abraham": 1,
    "Lincoln": 1,
    "A": 2,
    "na": 1,
    "_": 1
}

def calculate_precision_recall_f1(true_dict, pred_dict):
    tp = sum([min(pred_dict.get(entity, 0), count) for entity, count in true_dict.items()])
    fp = sum([max(pred_dict.get(entity, 0) - count, 0) for entity, count in true_dict.items()]) + sum([pred_dict.get(entity, 0) for entity in set(pred_dict.keys()) - set(true_dict.keys())])
    fn = sum([max(count - pred_dict.get(entity, 0), 0) for entity, count in true_dict.items()])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

chap_3_precision, chap_3_recall, chap_3_f1 = calculate_precision_recall_f1(chap_3_ground_truth, chapter3)
chap_4_precision, chap_4_recall, chap_4_f1 = calculate_precision_recall_f1(chap_4_ground_truth, chapter4)

print(f'Chapter 3 - Precision: {chap_3_precision}, Recall: {chap_3_recall}, F1: {chap_3_f1}')
print(f'Chapter 4 - Precision: {chap_4_precision}, Recall: {chap_4_recall}, F1: {chap_4_f1}')


Chapter 3 - Precision: 0.5714285714285714, Recall: 0.5, F1: 0.5333333333333333
Chapter 4 - Precision: 0.3695652173913043, Recall: 0.4473684210526316, F1: 0.40476190476190477


In [ ]:
import nltk
from bs4 import BeautifulSoup
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import requests
from nltk.tokenize import sent_tokenize

with open(file_path, 'r') as f:
    data = f.read()
chapters = data.split('CHAPTER')[1:]

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

num_topics = 10
num_words = 5

# split the text into sentences
sentences = sent_tokenize(data)

# clean each sentence
clean_sentences = [clean(sentence) for sentence in sentences]

# create the dictionary
dictionary = corpora.Dictionary(clean_sentences)

# create the document-term matrix
doc_term_matrix = [dictionary.doc2bow(sentence) for sentence in clean_sentences]

# run the LDA model
lda = LdaModel(doc_term_matrix, num_topics, id2word=dictionary, passes=50)

# print the topics
print(lda.print_topics(num_topics, num_words))

items = lda.print_topics(num_topics, num_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
